### Module Import and Auto-Reload

In [10]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os, sys

from datetime import datetime
from typing import Dict, Any
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))
print(parent_dir)

/home/ct-admin/Documents/Langgraph/00_AudioProject/02_Call_Center_AI_Agent


In [12]:
# from src.Agents.call_center_agent.data import get_client_data
# client_data = get_client_data(user_id='1489698')

In [13]:
# account_overview = client_data.get("account_overview")
# account_overview

In [14]:
# Import ALL agent modules to get their prompt templates
from src.Agents.call_center_agent.parameter_helper import prepare_parameters
from src.Agents.call_center_agent.call_scripts import ScriptManager, CallStep as ScriptCallStep

# Import agent prompt templates directly
from src.Agents.call_center_agent.step01_name_verification import NAME_VERIFICATION_PROMPT
from src.Agents.call_center_agent.step02_details_verification import DETAILS_VERIFICATION_PROMPT
from src.Agents.call_center_agent.step03_reason_for_call import REASON_FOR_CALL_PROMPT
from src.Agents.call_center_agent.step04_negotiation import NEGOTIATION_PROMPT
from src.Agents.call_center_agent.step05_promise_to_pay import PROMISE_TO_PAY_PROMPT
from src.Agents.call_center_agent.step06_debicheck_setup import DEBICHECK_SETUP_PROMPT
from src.Agents.call_center_agent.step07_payment_portal import PAYMENT_PORTAL_PROMPT
from src.Agents.call_center_agent.step08_subscription_reminder import SUBSCRIPTION_REMINDER_PROMPT
from src.Agents.call_center_agent.step09_client_details_update import CLIENT_DETAILS_UPDATE_PROMPT
from src.Agents.call_center_agent.step10_referrals import REFERRALS_PROMPT
from src.Agents.call_center_agent.step11_further_assistance import FURTHER_ASSISTANCE_PROMPT
from src.Agents.call_center_agent.step12_query_resolution import QUERY_RESOLUTION_PROMPT
from src.Agents.call_center_agent.step13_escalation import ESCALATION_PROMPT
from src.Agents.call_center_agent.step14_cancellation import CANCELLATION_PROMPT
from src.Agents.call_center_agent.step15_closing import CLOSING_PROMPT

In [15]:
# Mock client data for testing
MOCK_CLIENT_DATA = {
    "profile": {
        "user_id": "12345",
        "user_name": "TEST001",
        "client_info": {
            "title": "Mr",
            "first_name": "John",
            "client_full_name": "John Smith",
            "email_address": "john.smith@email.com",
            "id_number": "8001015001081",
            "contact": {"mobile": "0821234567"}
        },
        "vehicles": [{
            "registration": "ABC123GP",
            "make": "Toyota", 
            "model": "Corolla",
            "color": "Silver"
        }]
    },
    "account_aging": {
        "x0": 0,      # Current
        "x30": 199,   # 1-30 days overdue
        "x60": 0,     # 31-60 days
        "x90": 0,     # 61-90 days  
        "x120": 0,    # 90+ days
        "xbalance": 199
    },
    "account_overview": {
        "account_status": "Overdue",
        "cancellation_fee": 150
    },
    "subscription": {
        "subscription_amount": 199
    },
    "banking_details": {
        "bank_name": "FNB",
        "account_number": "1234567890"
    },
    "payment_history": []
}

# Mock state for different scenarios
MOCK_STATE = {
    "name_verification_status": "INSUFFICIENT_INFO",
    "name_verification_attempts": 1,
    "details_verification_status": "INSUFFICIENT_INFO", 
    "details_verification_attempts": 1,
    "field_to_verify": "ID number",
    "matched_fields": [],
    "call_outcome": "incomplete",
    "department": "Supervisor",
    "response_time": "24 hours",
    "last_client_question": "How does Cartrack work?",
    "should_offer_referrals": True,
    "current_step": "introduction"
}

In [16]:
AGENT_PROMPTS = {
    "step01_name_verification": NAME_VERIFICATION_PROMPT,
    "step02_details_verification": DETAILS_VERIFICATION_PROMPT,
    "step03_reason_for_call": REASON_FOR_CALL_PROMPT,
    "step04_negotiation": NEGOTIATION_PROMPT,
    "step05_promise_to_pay": PROMISE_TO_PAY_PROMPT,
    "step06_debicheck_setup": DEBICHECK_SETUP_PROMPT,
    "step07_payment_portal": PAYMENT_PORTAL_PROMPT,
    "step08_subscription_reminder": SUBSCRIPTION_REMINDER_PROMPT,
    "step09_client_details_update": CLIENT_DETAILS_UPDATE_PROMPT,
    "step10_referrals": REFERRALS_PROMPT,
    "step11_further_assistance": FURTHER_ASSISTANCE_PROMPT,
    "step12_query_resolution": QUERY_RESOLUTION_PROMPT,
    "step13_escalation": ESCALATION_PROMPT,
    "step14_cancellation": CANCELLATION_PROMPT,
    "step15_closing": CLOSING_PROMPT
}

In [17]:
def test_parameter_preparation():
    """Test parameter preparation for all scenarios"""
    print("🧪 Testing Parameter Preparation...")
    
    params = prepare_parameters(MOCK_CLIENT_DATA, MOCK_STATE, "Agent Smith")
    params["current_date"] = datetime.now().strftime("%Y-%m-%d %H:%M")
    
    print(f"📊 Generated {len(params)} parameters:")
    for key, value in sorted(params.items()):
        print(f"  {key}: {value}")
    
    return params

# Run test
test_params = test_parameter_preparation()

2025-06-02 12:47:50,406 - INFO - Selected RATIO_1_INFLOW: x30=199.0


🧪 Testing Parameter Preparation...
📊 Generated 47 parameters:
  account_status: Overdue
  agent_name: Agent Smith
  aging_category: First Missed Payment
  agreed_amount: R 0.00
  amount_with_fee: R 209.00
  banking_details: {'bank_name': 'FNB', 'account_number': '1234567890'}
  call_outcome: incomplete
  campaign_end_date: month-end
  campaign_first_date: next week
  cancellation_fee: R 150.00
  client_full_name: John Smith
  client_name: John
  client_title: Mr
  current_date: 2025-06-02 12:47
  current_email: john.smith@email.com
  current_mobile: 0821234567
  current_step: introduction
  department: Supervisor
  details_verification_attempts: 1
  details_verification_status: INSUFFICIENT_INFO
  discount_percentage: 50
  discounted_amount: R 119.40
  discounted_amount_50: R 99.50
  field_to_verify: ID number
  last_client_question: How does Cartrack work?
  matched_fields: []
  max_details_verification_attempts: 5
  max_name_verification_attempts: 5
  name_verification_attempts: 1
  

### Run Test

In [18]:
def test_all_agent_prompts():
    """Test all agent prompts using their actual templates"""
    print("\n🎭 Testing All Agent Prompts with Real Templates...")
    
    # Get script type and context
    account_aging = MOCK_CLIENT_DATA.get("account_aging", {})
    script_type = ScriptManager.determine_script_type_from_aging(account_aging, MOCK_CLIENT_DATA)
    aging_context = ScriptManager.get_aging_context(script_type)
    
    print(f"📋 Script Type: {script_type.value}")
    print(f"📊 Aging Context: {aging_context['category']} ({aging_context['urgency']})")
    
    # Enhanced params with aging context
    enhanced_params = test_params.copy()
    enhanced_params.update({
        "aging_approach": aging_context['approach'],
        "tone": aging_context['tone'],
        "aging_consequences": aging_context['consequences']
    })
    
    # Test each agent prompt
    results = {}
    
    for step_name, prompt_template in AGENT_PROMPTS.items():
        print(f"\n{'='*60}")
        print(f"🎯 Testing {step_name.upper()}")
        print(f"{'='*60}")
        
        try:
            script_step_name = step_name[7:]
            

            call_step = ScriptCallStep(script_step_name)
            script_template_content = ScriptManager.get_script_content(script_type, call_step)
            formatted_script = script_template_content.format(**enhanced_params) if script_template_content else f"No script for {script_step_name}"

            enhanced_params["formatted_script"] = formatted_script

            # Format the prompt
            formatted_prompt = prompt_template.format(**enhanced_params)
            
            # Store result
            results[step_name] = {
                "formatted_script": formatted_script,
                "formatted_prompt": formatted_prompt,
                "word_count": len(formatted_prompt.split())
            }
            
            print(f"📝 Script: {formatted_script[:100]}...")
            print(f"📄 Prompt Length: {len(formatted_prompt)} chars, {len(formatted_prompt.split())} words")
            print(f"💬 Formatted Prompt:")
            print(formatted_prompt)
            
        except Exception as e:
            print(f"❌ Error formatting {step_name}: {e}")
            results[step_name] = {"error": str(e)}
    
    return results

# Run test
all_prompt_results = test_all_agent_prompts()

2025-06-02 12:47:53,080 - INFO - Selected RATIO_1_INFLOW: x30=199.0



🎭 Testing All Agent Prompts with Real Templates...
📋 Script Type: ratio_1_inflow
📊 Aging Context: First Missed Payment (Medium)

🎯 Testing STEP01_NAME_VERIFICATION
📝 Script: No script for name_verification...
📄 Prompt Length: 1865 chars, 253 words
💬 Formatted Prompt:

<role>
You are debt collection specialist, named Agent Smith from Cartrack Accounts Department. 
Today time: 2025-06-02 12:47
</role>

<context>
Target client: John Smith | Outstanding amount: R 199.00 | Name Verification Status: INSUFFICIENT_INFO
Verification Attempt: 1/5| Urgency: Medium | Category: First Missed Payment | user_id: 12345
</context>

<script>
No script for name_verification
</script>

<task>
Confirm client identity. Match urgency to account severity.
</task>

<verification_responses>
**INSUFFICIENT_INFO** (Progressive approach based on urgency):
- Standard Urgency: "Hi, just to confirm I'm speaking with John Smith?"
- High Urgency: "This is urgent regarding your Cartrack account. Is this John Smith?"
- L

In [ ]:

from src.Agents.call_center_agent.data.client_data_fetcher import  get_client_data

client_data = get_client_data('23001', force_reload=True)

In [ ]:
# client_data

In [ ]:
# Determine script type from aging
account_aging = client_data.get("account_aging", {})
account_aging

In [ ]:
payment_history = client_data.get("payment_history", [])
# payment_history.get("arrangement_state")
payment_history

In [ ]:
from src.Agents.call_center_agent.call_scripts import ScriptManager
from src.Agents.call_center_agent.data.client_data_fetcher import  get_client_data

client_data = get_client_data('23001', force_reload=True)
account_aging = client_data.get("account_aging", {})
print(account_aging)

script_type = ScriptManager.determine_script_type_from_aging(account_aging, client_data)
aging_context = ScriptManager.get_aging_context(script_type)
script_type

In [ ]:
from src.Agents.call_center_agent.step00_introduction import get_introduction_prompt
from src.Agents.call_center_agent.step01_name_verification import get_name_verification_prompt
from src.Agents.call_center_agent.step02_details_verification import get_details_verification_prompt
from src.Agents.call_center_agent.step03_reason_for_call import get_reason_for_call_prompt
from src.Agents.call_center_agent.step04_negotiation import get_negotiation_prompt
from src.Agents.call_center_agent.step05_promise_to_pay import get_promise_to_pay_prompt
from src.Agents.call_center_agent.step06_debicheck_setup import get_debicheck_setup_prompt
from src.Agents.call_center_agent.step07_payment_portal import get_payment_portal_prompt
from src.Agents.call_center_agent.step08_subscription_reminder import get_subscription_reminder_prompt
from src.Agents.call_center_agent.step09_client_details_update import get_client_details_update_prompt
from src.Agents.call_center_agent.step10_referrals import get_referrals_prompt
from src.Agents.call_center_agent.step11_further_assistance import get_further_assistance_prompt
from src.Agents.call_center_agent.step12_query_resolution import get_query_resolution_prompt
from src.Agents.call_center_agent.step13_escalation import get_escalation_prompt
from src.Agents.call_center_agent.step14_cancellation import get_cancellation_prompt
from src.Agents.call_center_agent.step15_closing import get_closing_prompt
from app_config import CONFIG # Assuming CONFIG is used for some default values or agent_name

# client_data = {
#     'profile': {
#         'client_info': {
#             'title': 'Mr',
#             'first_name': 'Sampie',
#             'client_full_name': 'Sampie BLOEM'
#         }
#     },
#     'account_overview': {
#         'xbalance': '310.00',
#         'subscription': '179.00',
#         'cancellation_fee': '1969.00'
#     },
#     'account_aging': {
#         'xbalance': '310.00'
#     },
#     'payment_history': [
#         {'amount_paid': '100.00', 'payment_date': '2025-05-20', 'agreed_amount': '200.00', 'shortfall': '100.00'}
#     ]
# }

# state = {
#     "agent_name": "Sarah",
#     "field_to_verify": "ID number",
#     "amount_with_fee": 320.00,
#     "outcome_summary": "We've resolved your payment for today."
# }

agent_name = "AI Agent" # Or retrieve from CONFIG if available, e.g., CONFIG.get("agent_name", "AI Agent")
state = {} # Initialize an empty state or with relevant default values

# --- List of prompt functions ---
# You can create a list of tuples, where each tuple is (step_name, prompt_function)
prompt_functions = [
    ("Introduction", get_introduction_prompt),
    ("Name Verification", get_name_verification_prompt),
    ("Details Verification", get_details_verification_prompt),
    ("Reason for Call", get_reason_for_call_prompt),
    ("Negotiation", get_negotiation_prompt),
    ("Promise to Pay", get_promise_to_pay_prompt),
    ("Debicheck Setup", get_debicheck_setup_prompt),
    ("Payment Portal", get_payment_portal_prompt),
    ("Subscription Reminder", get_subscription_reminder_prompt),
    ("Client Details Update", get_client_details_update_prompt),
    ("Referrals", get_referrals_prompt),
    ("Further Assistance", get_further_assistance_prompt),
    ("Query Resolution", get_query_resolution_prompt),
    ("Escalation", get_escalation_prompt),
    ("Cancellation", get_cancellation_prompt),
    ("Closing", get_closing_prompt),
]

# --- Loop to export (print) each prompt ---
print("--- Exporting Call Center Prompts ---")
print("="*40)

for step_name, prompt_func in prompt_functions:
    try:
        # Call the prompt function with your data
        if step_name == "Introduction":
            prompt_content = prompt_func(client_data, agent_name, state)
        else:
            prompt_content = prompt_func(client_data, state)

        print(f"\n## {step_name} Prompt ##")
        print("-" * (len(step_name) + 10)) # Decorative line based on title length
        print(prompt_content)
        print("\n" + "="*40 + "\n") # Separator

    except Exception as e:
        print(f"\n## Error exporting {step_name} Prompt ##")
        print(f"An error occurred: {e}")
        print("\n" + "="*40 + "\n")

print("--- Prompt Export Complete ---")

### Test

In [ ]:
# ./example_usage_short.py
"""
Compact test to see full prompts for each call center step.
"""

from src.Agents.call_center_agent.call_scripts import ScriptType, CallStep
from src.Agents.call_center_agent.data_parameter_builder import (
    get_client_data,
    prepare_parameters, 
    ConversationState,
    VerificationStatus
)
from src.Agents.call_center_agent.prompts import get_step_prompt

def test_all_step_prompts():
    """Test all step prompts with full content display."""
    
    # Setup
    user_id = "83906"
    agent_name = "Sarah Mitchell"
    script_type = "ratio_1_inflow"
    
    print("🔄 Loading client data...")
    client_data = get_client_data(user_id)
    client_name = client_data.get('profile', {}).get('client_info', {}).get('client_full_name', 'Client')
    print(f"✅ Loaded data for: {client_name}")
    
    # Initialize state as fully verified
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    state.add_matched_field("id_number")
    state.add_objection("no_money")
    state.update_emotional_state("cooperative")
    
    # All steps to test
    steps = [
        "introduction",
        "name_verification", 
        "details_verification",
        "reason_for_call",
        "negotiation",
        "promise_to_pay",
        "debicheck_setup",
        "payment_portal",
        "subscription_reminder",
        "client_details_update",
        "referrals",
        "further_assistance",
        "query_resolution",
        "escalation",
        "cancellation",
        "closing"
    ]
    
    print(f"\n🧪 Testing {len(steps)} call center steps:")
    print("=" * 80)
    
    for i, step in enumerate(steps, 1):
        print(f"\n📋 STEP {i:2d}: {step.upper()}")
        print("-" * 50)
        
        try:
            # Build parameters
            parameters = prepare_parameters(
                client_data=client_data,
                current_step=step,
                state=state.to_dict(),
                script_type=script_type,
                agent_name=agent_name
            )
            
            # Generate full prompt
            full_prompt = get_step_prompt(step, parameters)
            
            # Display prompt (truncate if too long)
            if len(full_prompt) > 800:
                print(f"🎯 PROMPT: {full_prompt[:400]}...")
                print(f"...{full_prompt[-400:]}")
            else:
                print(f"🎯 PROMPT: {full_prompt}")
            
            # Show key parameters
            key_params = {
                "client_name": parameters.get("client_name", "N/A"),
                "outstanding_amount": parameters.get("outstanding_amount", "N/A"),
                "urgency_level": parameters.get("urgency_level", "N/A"),
                "key_motivators": parameters.get("key_motivators", "N/A")
            }
            print(f"📊 KEY PARAMS: {key_params}")
            
        except Exception as e:
            print(f"❌ ERROR: {e}")
        
        print("-" * 50)
    
    print(f"\n✅ Tested all {len(steps)} steps successfully!")

def test_script_variations_short():
    """Test different script types with key differences highlighted."""
    
    print("\n🎭 SCRIPT VARIATIONS TEST")
    print("=" * 60)
    
    user_id = "83906"
    agent_name = "Sarah"
    
    client_data = get_client_data(user_id)
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    script_types = [
        ("ratio_1_inflow", "First missed payment"),
        ("ratio_2_3_inflow", "2-3 months overdue"), 
        ("pre_legal_120_plus", "Pre-legal 120+ days"),
        ("pre_legal_150_plus", "Legal attorney involved")
    ]
    
    for script_type, description in script_types:
        print(f"\n📝 {description.upper()}")
        print("-" * 30)
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step="reason_for_call",
            state=state.to_dict(),
            script_type=script_type,
            agent_name=agent_name
        )
        
        # Show script content
        script_content = parameters.get("script_content", "No script content")
        print(f"💬 SCRIPT: {script_content}")
        
        # Show tactical differences
        urgency = parameters.get("urgency_level", "None")
        motivators = parameters.get("key_motivators", "None")
        
        print(f"⚡ URGENCY: {urgency}")
        print(f"🎯 MOTIVATORS: {motivators}")

def test_conversation_intelligence():
    """Test conversation intelligence features."""
    
    print("\n🧠 CONVERSATION INTELLIGENCE TEST")
    print("=" * 50)
    
    from src.Agents.call_center_agent.data_parameter_builder import (
        detect_emotional_state,
        detect_objections,
        analyze_payment_conversation
    )
    
    # Simulate conversation messages
    test_messages = [
        {"role": "ai", "content": "Can we debit R500 today?"},
        {"role": "human", "content": "I don't have money right now, I'm really stressed about this"},
        {"role": "ai", "content": "I understand. What amount could you manage?"},
        {"role": "human", "content": "Maybe R200 next week when I get paid"}
    ]
    
    print("💭 Test conversation:")
    for msg in test_messages:
        speaker = "AGENT" if msg["role"] == "ai" else "CLIENT"
        print(f"   {speaker}: {msg['content']}")
    
    # Analyze conversation
    emotional_state = detect_emotional_state(test_messages)
    objections = detect_objections(test_messages)
    payment_analysis = analyze_payment_conversation(test_messages, 500.0)
    
    print(f"\n🎭 EMOTIONAL STATE: {emotional_state}")
    print(f"🚫 OBJECTIONS: {objections}")
    print(f"💰 PAYMENT ANALYSIS: {payment_analysis}")

def test_outstanding_calculation():
    """Test outstanding amount calculation."""
    
    print("\n💰 OUTSTANDING AMOUNT CALCULATION TEST")
    print("=" * 50)
    
    from src.Agents.call_center_agent.data_parameter_builder import (
        calculate_outstanding_amount,
        format_outstanding_amount
    )
    
    # Test data
    test_aging_data = {
        "x0": "100.00",      # Current (not overdue)
        "x30": "200.00",     # 1-30 days overdue
        "x60": "150.00",     # 31-60 days overdue
        "x90": "100.00",     # 61-90 days overdue
        "x120": "50.00",     # 91+ days overdue
        "xbalance": "600.00" # Total balance
    }
    
    outstanding_float = calculate_outstanding_amount(test_aging_data)
    formatted_amount = format_outstanding_amount({"account_aging": test_aging_data})
    
    print(f"📊 AGING BREAKDOWN:")
    print(f"   Current (x0): R {test_aging_data['x0']}")
    print(f"   30 days: R {test_aging_data['x30']}")
    print(f"   60 days: R {test_aging_data['x60']}")
    print(f"   90 days: R {test_aging_data['x90']}")
    print(f"   120+ days: R {test_aging_data['x120']}")
    print(f"   TOTAL: R {test_aging_data['xbalance']}")
    
    print(f"\n💯 CALCULATION:")
    print(f"   Outstanding = Total - Current")
    print(f"   Outstanding = R {test_aging_data['xbalance']} - R {test_aging_data['x0']}")
    print(f"   Outstanding = R {outstanding_float:.2f}")
    print(f"   Formatted: {formatted_amount}")

def quick_test():
    """Quick test of key functionality."""
    
    print("🚀 QUICK CALL CENTER TEST")
    print("=" * 40)
    
    # Test 3 key steps
    key_steps = ["reason_for_call", "negotiation", "promise_to_pay"]
    
    user_id = "83906"
    client_data = get_client_data(user_id)
    state = ConversationState()
    state.name_verification_status = VerificationStatus.VERIFIED.value
    state.details_verification_status = VerificationStatus.VERIFIED.value
    
    for step in key_steps:
        print(f"\n🎯 {step.upper()}")
        
        parameters = prepare_parameters(
            client_data=client_data,
            current_step=step,
            state=state.to_dict(),
            script_type="ratio_1_inflow",
            agent_name="Sarah"
        )
        
        prompt = get_step_prompt(step, parameters)
        
        # Show first 200 chars of prompt
        print(f"📝 PROMPT: {prompt[:200]}...")
        
        # Show key fields
        outstanding = parameters.get("outstanding_amount", "N/A")
        urgency = parameters.get("urgency_level", "N/A")
        print(f"💰 Amount: {outstanding} | ⚡ Urgency: {urgency}")

if __name__ == "__main__":
    print("🎯 CALL CENTER SYSTEM - COMPACT TESTS")
    print("=" * 60)
    
    # Choose which test to run
    test_choice = input("\nChoose test:\n1. All step prompts\n2. Script variations\n3. Conversation intelligence\n4. Outstanding calculation\n5. Quick test\n\nEnter number (1-5): ")
    
    if test_choice == "1":
        test_all_step_prompts()
    elif test_choice == "2":
        test_script_variations_short()
    elif test_choice == "3":
        test_conversation_intelligence()
    elif test_choice == "4":
        test_outstanding_calculation()
    elif test_choice == "5":
        quick_test()
    else:
        print("Running quick test by default...")
        quick_test()
    
    print("\n✅ Test completed!")

In [ ]:
user_id = "83906"
agent_name = "Sarah Mitchell"
script_type = "ratio_1_inflow"  # Standard first missed payment
client_data = get_client_data(user_id)


In [ ]:
state = ConversationState()
state.add_objection("no_money")
state.update_emotional_state("defensive")
state.update_payment_willingness("low")
parameters = prepare_parameters(
        client_data=client_data,
        current_step="negotiation",
        state=state.to_dict(),
        script_type=script_type,
        agent_name=agent_name
    )
    
reason_prompt = get_step_prompt("reason_for_call", parameters)
print(reason_prompt)